# R2 Overview:

Remember to do:

Setup

List functions, then print code for one

In [1]:
%pip install r2pipe

Note: you may need to restart the kernel to use updated packages.


In [1]:
import r2pipe

rp = r2pipe.open('loop2', flags=['-d'])

Process with PID 4463 started...
= attach 4463 4463
bin.baddr 0x55668361c000
Using 0x55668361c000
asm.bits 64


In [4]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)
@register_line_magic
def r(line):
    print(line)
    print()
    print(rp.cmd(line))
    print(rp.cmd(""))
    return

In [5]:
%r aa;e asm.syntax=att; afl;

aa;e asm.syntax=att; afl;

0x55668361c4f0    1 42           entry0
0x55668381cfe0    1 4124         reloc.__libc_start_main
0x55668361c520    4 50   -> 40   sym.deregister_tm_clones
0x55668361c560    4 66   -> 57   sym.register_tm_clones
0x55668361c5b0    5 58   -> 51   entry.fini0
0x55668361c4e0    1 6            sym.imp.__cxa_finalize
0x55668361c5f0    1 10           entry.init0
0x55668361c6b0    1 2            sym.__libc_csu_fini
0x55668361c6b4    1 9            sym._fini
0x55668361c640    4 101          sym.__libc_csu_init
0x55668361c5fa    4 66           main
0x55668361c4b8    3 23           sym._init

0x55668361c4f0    1 42           entry0
0x55668381cfe0    1 4124         reloc.__libc_start_main
0x55668361c520    4 50   -> 40   sym.deregister_tm_clones
0x55668361c560    4 66   -> 57   sym.register_tm_clones
0x55668361c5b0    5 58   -> 51   entry.fini0
0x55668361c4e0    1 6            sym.imp.__cxa_finalize
0x55668361c5f0    1 10           entry.init0
0x55668361c6b0    1 2       

In [6]:
%r pdf @main

pdf @main


            ; DATA XREF from entry0 @ 0x55668361c50d
┌ 66: int main (int argc, char **argv, char **envp);
│           ; var int64_t var_ch @ rbp-0xc
│           ; var int64_t var_8h @ rbp-0x8
│           ; var int64_t var_4h @ rbp-0x4
│           0x55668361c5fa      55             pushq %rbp
│           0x55668361c5fb      4889e5         movq %rsp, %rbp
│           0x55668361c5fe      c745f4140000.  movl $0x14, var_ch      ; 20
│           0x55668361c605      c745f8160000.  movl $0x16, var_8h      ; 22
│           0x55668361c60c      c745fc000000.  movl $0, var_4h
│           0x55668361c613      c745fc040000.  movl $4, var_4h
│       ┌─< 0x55668361c61a      eb13           jmp 0x55668361c62f
│      ┌──> 0x55668361c61c      8365f402       andl $2, var_ch
│      ╎│   0x55668361c620      d17df8         sarl $1, var_8h
│      ╎│   0x55668361c623      8b55fc         movl var_4h, %edx
│      ╎│   0x55668361c626      89d0           movl %edx, %eax
│      ╎│   0x55668361c628      01

# Other Notes:

# R2: Set Break Points:


In [15]:
%r db 0x55668361c63a

db 0x55668361c63a





In [18]:
%r dc

dc





hit breakpoint at: 55668361c63a


In [19]:
%r pdf @main

pdf @main


            ; DATA XREF from entry0 @ 0x55668361c50d
┌ 66: int main (int argc, char **argv, char **envp);
│           ; var int64_t var_ch @ rbp-0xc
│           ; var int64_t var_8h @ rbp-0x8
│           ; var int64_t var_4h @ rbp-0x4
│           0x55668361c5fa      55             pushq %rbp
│           0x55668361c5fb      4889e5         movq %rsp, %rbp
│           0x55668361c5fe      c745f4140000.  movl $0x14, var_ch      ; 20
│           0x55668361c605      c745f8160000.  movl $0x16, var_8h      ; 22
│           0x55668361c60c      c745fc000000.  movl $0, var_4h
│           0x55668361c613      c745fc040000.  movl $4, var_4h
│       ┌─< 0x55668361c61a      eb13           jmp 0x55668361c62f
│      ┌──> 0x55668361c61c      8365f402       andl $2, var_ch
│      ╎│   0x55668361c620      d17df8         sarl $1, var_8h
│      ╎│   0x55668361c623      8b55fc         movl var_4h, %edx
│      ╎│   0x55668361c626      89d0           movl %edx, %eax
│      ╎│   0x55668361c628      01

In [21]:
%r px @ rbp-0x8

px @ rbp-0x8


- offset -       0 1  2 3  4 5  6 7  8 9  A B  C D  E F  0123456789ABCDEF
0x7fff8c567578  0200 0000 6c00 0000 40c6 6183 6655 0000  ....l...@.a.fU..
0x7fff8c567588  ca5c 6ea3 bc7f 0000 7876 568c ff7f 0000  .\n.....xvV.....
0x7fff8c567598  0000 0000 0100 0000 fac5 6183 6655 0000  ..........a.fU..
0x7fff8c5675a8  d957 6ea3 bc7f 0000 0000 0000 0000 0000  .Wn.............
0x7fff8c5675b8  52db 23b4 1801 e0cd f0c4 6183 6655 0000  R.#.......a.fU..
0x7fff8c5675c8  0000 0000 0000 0000 0000 0000 0000 0000  ................
0x7fff8c5675d8  0000 0000 0000 0000 52db 83d3 771f d298  ........R...w...
0x7fff8c5675e8  52db a581 0741 5498 0000 0000 0000 0000  R....AT.........
0x7fff8c5675f8  0000 0000 0000 0000 0000 0000 0000 0000  ................
0x7fff8c567608  0100 0000 0000 0000 7876 568c ff7f 0000  ........xvV.....
0x7fff8c567618  8876 568c ff7f 0000 8091 8ca3 bc7f 0000  .vV.............
0x7fff8c567628  0000 0000 0000 0000 0000 0000 0000 0000  ................
0x7fff8c567638  f0c4 61

In [10]:
%r ds

ds




In [10]:
%r dc
%r px @rbp-0x4
%r ds

dc

rax = 0xffffffffffffffda
rbx = 0x7fdb70d3b610
rcx = 0x7fdb70c46659
rdx = 0x0000003c
r8 = 0xffffffffffffff80
r9 = 0x00000001
r10 = 0x00000001
r11 = 0x00000246
r12 = 0x7fdb70d3b610
r13 = 0x00000001
r14 = 0x7fdb70d3bae8
r15 = 0x00000000
rsi = 0x000000e7
rdi = 0x00000000
rsp = 0x7fff9e019e68
rbp = 0x00000000
rip = 0x7fdb70c46659
rflags = 0x00000246
orax = 0x000000e7

px @rbp-0x4


ds

- offset -           0 1  2 3  4 5  6 7  8 9  A B  C D  E F  0123456789ABCDEF
0xfffffffffffffffc  ffff ffff ffff ffff ffff ffff ffff ffff  ................
0x0000000c  ffff ffff ffff ffff ffff ffff ffff ffff  ................
0x0000001c  ffff ffff ffff ffff ffff ffff ffff ffff  ................
0x0000002c  ffff ffff ffff ffff ffff ffff ffff ffff  ................
0x0000003c  ffff ffff ffff ffff ffff ffff ffff ffff  ................
0x0000004c  ffff ffff ffff ffff ffff ffff ffff ffff  ................
0x0000005c  ffff ffff ffff ffff ffff ffff ffff ffff  ................
0x0000006c  ffff ffff ffff ffff ffff

child exited with status 0

==> Process finished

trace_pc: cannot get opcode size at 0x7fdb70c46659
Step failed
